In [1]:
import numpy as np
import matplotlib.pyplot as plt
import cv2
import glob
import pickle
import visutils
import camutils as cam
import selectpoints
import meshutils
import matplotlib.patches as patches
from mpl_toolkits.mplot3d import Axes3D
import scipy
import trimesh
from collections import defaultdict

In [2]:
# Mesh cleanup parameters
# Specify limits along the x,y and z axis of a box containing the object
# we will prune out triangulated points outside these limits
def box_limit(boxlimit, pts3, pts2L, pts2R):
    index =np.where((pts3[0,:] <= boxlimit[0]))[0]
    index = np.concatenate((index, np.where((pts3[0,:] >= boxlimit[1]))[0]))
    index = np.concatenate((index, np.where((pts3[1,:] <= boxlimit[2]))[0]))
    index = np.concatenate((index, np.where((pts3[1,:] >= boxlimit[3]))[0]))
#     print(boxlimit[3])
    index = np.concatenate((index, np.where((pts3[2,:] <= boxlimit[4]))[0]))
    index = np.concatenate((index, np.where((pts3[2,:] >= boxlimit[5]))[0]))
    pts3 = np.delete(pts3,index, axis = 1)
    pts2L = np.delete(pts2L,index, axis = 1)
    pts2R = np.delete(pts2R,index, axis = 1)
    return pts3,pts2L,pts2R


# Specify a longest allowed edge that can appear in the mesh. Remove triangles
# from the final mesh that have edges longer than this value
def triangle_pruning(tri, trithresh, pts3, pts2L, pts2R):
    
    pts3_triangles = pts3.T[tri.simplices]
    badtriangles = []
    for x in range(len(pts3_triangles)):
        row1 = pts3_triangles[x, 0]
        row2 =  pts3_triangles[x, 1]
        row3 =  pts3_triangles[x, 2]
        if (np.linalg.norm(row1-row2) >= trithresh or np.linalg.norm(row1-row3) >= trithresh or np.linalg.norm(row2-row3) >= trithresh):
                   badtriangles.append(x)
    return np.delete(tri.simplices, badtriangles, 0)
#average out the length 
def average(vertex, vertex_neighbors, vertices):
    sumx, sumy, sumz = 0, 0, 0
    for x in vertex_neighbors:
        sumx += vertices[x][0]
        sumy += vertices[x][1]
        sumz += vertices[x][2]
    return [sumx/len(vertex_neighbors), sumy/len(vertex_neighbors), sumz/len(vertex_neighbors)]

#return color array
def getColor(pts2L, num):
    imgLColor = plt.imread("couple/grab_"+str(num)+"_u/color_C0_01.png")
    color = []
    for x in pts2L.T:
            color.append(imgLColor[x[1],x[0]])
    return np.array(color)
    
def mesh(boxlimit, trithresh, pts3, pts2L, pts2R, num ):
    indices = []
    pts3, pts2L, pts2R = box_limit(boxlimit, pts3, pts2L,pts2R)
    tri = scipy.spatial.Delaunay(pts2L.T)
    tri.simplices = triangle_pruning(tri, trithresh, pts3, pts2L, pts2R)
    color = getColor(pts2L, num)
    facecolor = trimesh.visual.color.vertex_to_face_color(color, tri.simplices)
    mesh = trimesh.Trimesh(vertices=pts3.T,faces=tri.simplices[:,[0,2,1]],face_colors = facecolor,  vertex_colors = color)
    for a in range(4):
        pts3_new = []
        for index, x in enumerate(mesh.vertices):
            pts3_new.append(average(x, mesh.vertex_neighbors[index], mesh.vertices))
#         print(a)
        mesh.vertices = pts3_new
    for index, x in enumerate(pts3.T):
        if x not in mesh.vertices:
            indices.append(index)
    pts3 = np.delete(pts3,indices, axis = 1)
    pts2L = np.delete(pts2L,indices, axis = 1)
    pts2R = np.delete(pts2R,indices, axis = 1)
    return mesh

In [3]:
fid = open('pts_value.pickle','rb')
results = pickle.load(fid)
new_results = dict()
fid.close()
mesh0 = mesh(np.array([-15,25,0,30,-25,-15]), 1.6, results[0]["pts3"], results[0]["pts2L"],results[0]["pts2R"], 0)
mesh1 = mesh(np.array([-15,25,0,20,-40,0]), 1.6, results[1]["pts3"], results[1]["pts2L"],results[1]["pts2R"], 1)
mesh2 = mesh(np.array([-15,25,0,20,-30,-10]), 1.6, results[2]["pts3"], results[2]["pts2L"],results[2]["pts2R"],2 )
mesh3 = mesh(np.array([-15,25,-2,20,-30,-10]), 1.6, results[3]["pts3"], results[3]["pts2L"],results[3]["pts2R"], 3)
mesh4 = mesh(np.array([-10,25,0,20,-20,0]), 1.6, results[4]["pts3"], results[4]["pts2L"],results[4]["pts2R"], 4)
mesh6 = mesh(np.array([12.5,25,0,30,-30,0]), 1.6, results[6]["pts3"], results[6]["pts2L"],results[6]["pts2R"],6)


In [4]:
mesh0.export('mesh0.ply')
mesh1.export('mesh1.ply')
mesh2.export('mesh2.ply')
mesh3.export('mesh3.ply')
mesh4.export('mesh4.ply')
mesh6.export('mesh6.ply')

b'ply\nformat binary_little_endian 1.0\ncomment https://github.com/mikedh/trimesh\nelement vertex 15442\nproperty float x\nproperty float y\nproperty float z\nproperty uchar red\nproperty uchar green\nproperty uchar blue\nproperty uchar alpha\nelement face 30518\nproperty list uchar int vertex_indices\nend_header\n0\xbedAC\xf7\xee@\xd9$\xb8\xc1\x86\x84N\xff\x987dAE\\\xf5@`\x14\xb7\xc1\x94\x8bT\xff\xbb\x03dA\x90f\xf7@\xc5\x06\xb7\xc1\x94\x87S\xff\x03GdA\x9d\x1c\xfb@;n\xb6\xc1\x88\x83K\xff\xe6\xb1eA\xd6\x15\x00A8\xfc\xb4\xc1uq9\xff\xc1\xa6dA\xb6f\xf7@\xeeD\xb6\xc1\x97\x8f]\xfftMdA\x16\xe1\xf8@\x11f\xb6\xc1\x99\x90^\xffoUdA\xb2\xdd\xfb@\x04\xfe\xb5\xc1\x98\x90_\xffXWdA\xcbD\xfe@*\xc7\xb5\xc1\x92\x8aW\xff\xc5\xe5dA\xc4)\x00A\xf85\xb5\xc1\x8c\x83Q\xff\x1d\x97fAu\xca\x02A\xccw\xb3\xc1}s<\xffQaeA\xe4\xec\xe7@\xdc\xa6\xb8\xc1\x9a\x96h\xff\x95\x88eA\xcf\x01\xeb@\x12\xa4\xb7\xc1\x9f\x94n\xff!UeA"w\xee@[@\xb7\xc1\x9d\x95l\xff\xb6\xe3dA2\x9d\xf3@d\xbc\xb6\xc1\x9a\x92n\xff:ieA63\x00A-\xb4\xb4\xc1\x